Beat-n game with 10 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JLD2

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const MXV = 9
const MNV = 1

1

In [4]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [5]:


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [7]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [8]:
lib0 = get_library(10)
lib0

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [9]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [10]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [11]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end
nlib = length(lib)

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

74

In [12]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [13]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [14]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));

In [15]:
libi = [spud2int(ff) for ff in lib];

In [16]:
int2ind[313111]

73

In [17]:
as = [1,2,3]
as[end]

3

In [18]:
function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]==1
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

eval_beatn (generic function with 1 method)

In [19]:
as = [int2ind[ff] for ff in [214111, 112222, 313111, 113311]]
bs = [int2ind[ff] for ff in [121411, 121321, 113212, 213121]]

4-element Vector{Int64}:
 41
 40
 27
 65

In [20]:
eval_beatn(payoffs, as[1:3], bs[1:2])

2

In [21]:
eval_beatn(payoffs, bs[1:3], as[1:3])

2

In [22]:
eval_beatn(payoffs, as[1:4], bs[1:3])

2

In [23]:
eval_beatn(payoffs, bs[1:4], as[1:4])

3

In [24]:
size(payoffs)[1]

74

In [25]:
# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

try_beatn (generic function with 1 method)

In [26]:
for i in try_beatn(payoffs, as, bs)
    print(i)
end

00000000000000000000000011120110021211220011100001110000000000000120000000

In [27]:
# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

lose_or_not (generic function with 1 method)

In [28]:
lose_or_not(payoffs, as[1:2], bs[1:2])

0

In [29]:
lose_or_not(payoffs, as[1:3], bs[1:3])

0

In [30]:
lose_or_not(payoffs, as[1:4], bs[1:4])

1

In [31]:
function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

legal_moves (generic function with 1 method)

In [32]:
# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

choose_random (generic function with 1 method)

In [33]:
# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

choose_aggro (generic function with 1 method)

In [128]:
function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function play_random_vs_random(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_random(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end

display_game (generic function with 1 method)

In [129]:
a_win, as, bs = play_random_vs_aggro(payoffs, [int2ind[111133]], [int2ind[222211]])
display_game(payoffs, as, bs)

111133 {0}. 222211 {1}. 
112222 {1}. 111313 {2}. 
121321 {2}. 213112 {3}. 
213121 {3}. 


In [130]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# for a in 1:nlib
#     for b in 1:nlib
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a], [b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [131]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)
#     js = findall(wins[i, :].==minimum(wins[i, :]))
#     print(libi[i])
#     print(" ")
#     print(libi[js])
#     print(" ")
#     println(minimum(wins[i, :])/n_games)
# end

In [132]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a_moves = legal_moves(payoffs, [a1], [b1])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1], [a1, a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a], [b1,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [133]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [134]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2],[b1,b2])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
113131 {1}. 113311 {3}. 
132112 {3}. 131311 {3}. 
141112 {4}. 131212 {4}. 


In [135]:
# n_games = 1000
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a_moves = legal_moves(payoffs, [a1,a2], [b1,b2])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2], [a1,a2,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a], [b1,b2,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [136]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [137]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3],[b1,b2,b3])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
131311 {0}. 113311 {4}. 


In [138]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a3 = int2ind[121321]
# b3 = int2ind[121222]
# a_moves = legal_moves(payoffs, [a1,a2,a3], [b1,b2,b3])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2,b3], [a1,a2,a3,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a], [b1,b2,b3,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [139]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [144]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
#a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
a_win, as, bs = play_random_vs_random(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111133 {2}. 
111241 {2}. 122221 {2}. 
121132 {2}. 222112 {2}. 
112321 {2}. 114112 {4}. 


## MCTS

In [145]:
function intarr_to_string(as::Array{Int64})::String
    return join([convert(Char, a+32) for a in as])
end

function string_to_intarr(s::String)::Array{Int64}
    return [convert(Int64, s[i])-32 for i in 1:length(s)]
end

# s=intarr_to_string([i for i in 1:75])
# as=string_to_intarr(s)
# for i in as
#     print(i)
#     print(" ")
# end

function split_arrs(zs::Array{Int64})::Tuple{Array{Int64},Array{Int64}}
    if mod(length(zs), 2)==0
        return (zs[1:2:(end-1)], zs[2:2:end])
    else
        return (zs[1:2:end], zs[2:2:(end-1)])
    end
end

function merge_arrs(as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    zs = Array{Int64}(undef, length(as)+length(bs))
    if mod(length(zs), 2)==0
        zs[1:2:(end-1)]=as
        zs[2:2:end]=bs
    else
        zs[1:2:end] = as
    zs[2:2:(end-1)] = bs
    end
    return zs
end

merge_arrs (generic function with 1 method)

In [184]:
function propagate_win(tree::Dict{String, Vector{Int64}}, leaf::String, win::Int64)::Dict{String, Vector{Int64}}
    for i in 0:length(leaf)
        if i==0
            node=""
        else
            node=leaf[1:i]
        end
        if haskey(tree, node)
            tree[node][2] += 1
            tree[node][1] += win
        end
    end
    return tree
end

function append_move_to_leaf(leaf::String, move::Int64)::String
    zs = string_to_intarr(leaf)
    append!(zs, move)
    return intarr_to_string(zs)
end

function get_children(payoffs::Array{Int64}, leaf::String)::Array{String}
    # get arrays
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate successors
    if player==1
        cands = legal_moves(payoffs, as, bs)
    else
        cands = legal_moves(payoffs, bs, as)
    end
    return [append_move_to_leaf(leaf, c) for c in cands]
end

get_children (generic function with 1 method)

In [185]:
function check_and_propagate_terminal(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    children = get_children(payoffs, leaf)
    if length(children) == 0
        # indicate terminal
        tree[leaf][3]=1
        if mod(length(leaf), 2)==0
            #p1 lost
            tree = propagate_win(tree, leaf, 0)
        else
            #p1 won
            tree = propagate_win(tree, leaf, 1)
        end
    end
    return tree
end

check_and_propagate_terminal (generic function with 1 method)

In [186]:
function try_add_node(tree::Dict{String, Vector{Int64}}, node::String)::Dict{String, Vector{Int64}}
    if haskey(tree, node)
        return tree
    else
        tree[node] = [0,0,0]
        return tree
    end
end

function query_node(tree, node)::Array{Int64}
    if haskey(tree, node)
        return tree[node]
    else
        return [0,0,0]
    end
end

query_node (generic function with 1 method)

In [187]:
function expand_leaf(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    # check if leaf is terminal
    tree = check_and_propagate_terminal(payoffs, tree, leaf)
    if tree[leaf][3]==1
        return tree
    end
    # get arrays for leaf
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate random successor
    if player==1
        cands = legal_moves(payoffs, as, bs)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    else
        cands = legal_moves(payoffs, bs, as)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    end
    tree = try_add_node(tree, newleaf)
    tree = check_and_propagate_terminal(payoffs, tree, newleaf)
    # play the match
    as, bs = split_arrs(string_to_intarr(newleaf))
    a_win, as2, bs2 = play_random_vs_random(payoffs, as, bs)
    # propagate result
    if a_win == 1
        tree = propagate_win(tree, newleaf, 1)
    else
        tree = propagate_win(tree, newleaf, 0)
    end
    return tree
end


expand_leaf (generic function with 1 method)

In [188]:
function sample_next_node(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, 
        explore_const::Float64=1.4)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if minimum(counts) == 0
        sel = children[rand(findall(counts .== 0))]
        return sel
    end
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i] + explore_const * sqrt((log(n)/counts[i]))
        end
    end
    sel = children[rand(findall(scores .== maximum(scores)))]
    return sel
end

sample_next_node (generic function with 2 methods)

In [189]:
function random_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String="")::String
    while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
        node = sample_next_node(payoffs, tree, node)
    end
    return node
end

random_leaf (generic function with 2 methods)

In [190]:
# exploit mode
function sample_next_node2(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, beta::Float64 = 1.0)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i]
        end
    end
    scores = [exp(beta*x) for x in scores]
    scores = scores./sum(scores)
    cs = cumsum(scores)
    z = rand(1)[1]
    sel = children[minimum(findall(cs .>= z))]
    return sel
end

sample_next_node2 (generic function with 2 methods)

In [246]:
function display_node(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    print("Node: ")
    println(node)
    as, bs = split_arrs(string_to_intarr(node))
    display_game(payoffs, as, bs)
    print("Stats: ")
    print(query_node(tree,node)[1])
    print("/")
    print(query_node(tree,node)[2])
    print("=")
    s = string(query_node(tree,node)[1]/query_node(tree,node)[2])
    println(s[1:min(5, length(s))])
    if query_node(tree,node)[3]==1
        println("[terminal] ")
    end
end

function display_path(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    for i in 0:length(leaf)
        if i==0
            if haskey(tree, "")
                display_node(payoffs, tree, "")
                println()
            end
        else
            if haskey(tree, leaf[1:i])
                display_node(payoffs, tree, leaf[1:i])
                println()
            end
        end
    end
end

function display_choices(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    children = get_children(payoffs, node)
    display_node(payoffs, tree, node)
    for c in children
        println()
        display_node(payoffs, tree, c)
    end
end

display_choices (generic function with 1 method)

In [247]:
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
s0=intarr_to_string(merge_arrs([a1,a2,a3,a4],[b1,b2,b3,b4]))

"Hc`iHE9\""

In [248]:
function create_tree(node::String = "")::Dict{String, Vector{Int64}}
    return Dict(node => [0,0,0])
end

create_tree (generic function with 2 methods)

In [249]:
#tree = load_object("beatn10_tree.jld2")

tree = create_tree()


Dict{String, Vector{Int64}} with 1 entry:
  "" => [0, 0, 0]

In [250]:
#tree = expand_leaf(payoffs, tree, "")

In [251]:
node = random_leaf(payoffs, tree, "")

""

In [252]:
function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String = "")::Dict{String, Vector{Int64}}
    node = random_leaf(payoffs, tree, node)
    tree = try_add_node(tree, node)
    tree = check_and_propagate_terminal(payoffs, tree, node)
    tree = expand_leaf(payoffs, tree, node)
    return tree
end

function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}; 
        node::String = "", iters::Int64 = 1000)::Dict{String, Vector{Int64}}
    for i in 1:iters
        tree = grow_tree(payoffs, tree, node)
    end
    return tree
end

grow_tree (generic function with 3 methods)

In [253]:
@time tree = grow_tree(payoffs, tree, iters=100)

  0.068840 seconds (556.72 k allocations: 42.565 MiB, 20.72% compilation time)


Dict{String, Vector{Int64}} with 173 entries:
  "\"i" => [1, 1, 0]
  "Z"   => [0, 1, 0]
  "1"   => [0, 1, 0]
  "E?"  => [0, 1, 0]
  "="   => [0, 1, 0]
  "'"   => [0, 1, 0]
  "O7"  => [1, 1, 0]
  "\$Z" => [0, 1, 0]
  "NP"  => [0, 1, 0]
  "*"   => [0, 1, 0]
  ")"   => [2, 2, 0]
  "K"   => [0, 1, 0]
  ":e"  => [1, 1, 0]
  "<b"  => [1, 1, 0]
  "]P"  => [1, 1, 0]
  "hT"  => [0, 1, 0]
  "T-"  => [0, 1, 0]
  "4^"  => [1, 1, 0]
  "9Q"  => [0, 1, 0]
  "a4"  => [0, 1, 0]
  "9"   => [1, 2, 0]
  "F"   => [1, 2, 0]
  "c"   => [0, 1, 0]
  "HH"  => [0, 1, 0]
  "&U"  => [0, 1, 0]
  ⋮     => ⋮

In [254]:
# for metaiter in 1:10
#     s = ""
#     for iter in 1:100000
#         s = string(s,".")
#         node = random_leaf(payoffs, tree, "")
#         tree = try_add_node(tree, node)
#         tree = check_and_propagate_terminal(payoffs, tree, node)
#         s = string(s, node, ".")
#         tree = expand_leaf(payoffs, tree, node)
#     end
#     println(s[end-100:end])
#     save_object("beatn10_tree.jld2", tree)
# end

In [255]:
length(tree)

173

In [256]:
node = random_leaf(payoffs, tree, "")

"8"

In [257]:
# for mj in 1:2
#     nodes = Array{String}(undef, 50)
#     for i in 1:length(nodes)
#         node = ""
#         for j in 1:mj
#             if haskey(tree, node)
#                 node = sample_next_node2(payoffs, tree, node, 10.0)
#             end
#         end
#         nodes[i] = node
#     end
#     nodes = sort(unique(nodes))
#     for node in nodes
#         display_node(payoffs, tree, node)
#         println()
#     end
# end

In [258]:
# node = random_leaf(payoffs, tree, "")
# display_path(payoffs, tree, node)

In [259]:
# node = ""
# while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
#     node = sample_next_node(payoffs, tree, node)
#     println(node)
# end

In [260]:
# display_node(payoffs, tree, node)

In [261]:
# children = get_children(payoffs, node)
# children

In [262]:
# sample_next_node(payoffs, tree, node)

In [263]:
#display_choices(payoffs, tree, "HG`i\"")

### Tree at arbitrary root

In [268]:
length(s0)

8

In [272]:
node0 = s0[1:7]
tree = create_tree(node0)
@time tree = grow_tree(payoffs, tree, node0, 10000)

  6.875568 seconds (45.72 M allocations: 4.875 GiB, 54.34% gc time)


Dict{String, Vector{Int64}} with 17015 entries:
  "Hc`iHE9IWj#"   => [0, 1, 0]
  "Hc`iHE9>+cT"   => [1, 2, 1]
  "Hc`iHE9<+"     => [0, 2, 0]
  "Hc`iHE9V`"     => [1, 1, 0]
  "Hc`iHE9\$e5"   => [1, 1, 0]
  "Hc`iHE9:Y#"    => [1, 1, 0]
  "Hc`iHE9?&L"    => [1, 1, 0]
  "Hc`iHE9:&F"    => [1, 1, 0]
  "Hc`iHE9`7\\"   => [0, 1, 0]
  "Hc`iHE9\"g>e4" => [1, 1, 0]
  "Hc`iHE9\"4>J"  => [1, 1, 0]
  "Hc`iHE9H\\>\"" => [0, 1, 0]
  "Hc`iHE9UBf"    => [1, 1, 0]
  "Hc`iHE9bd["    => [1, 1, 0]
  "Hc`iHE9\"hcC"  => [1, 2, 0]
  "Hc`iHE9W<"     => [1, 1, 0]
  "Hc`iHE9EM("    => [1, 1, 0]
  "Hc`iHE9\"iT^"  => [0, 3, 0]
  "Hc`iHE9)+%_"   => [1, 1, 0]
  "Hc`iHE9[R\"]"  => [0, 1, 0]
  "Hc`iHE9\"F<c"  => [0, 1, 0]
  "Hc`iHE9@E"     => [2, 2, 0]
  "Hc`iHE9P:8"    => [0, 1, 0]
  "Hc`iHE9LI/"    => [1, 1, 0]
  "Hc`iHE9(*2"    => [2, 2, 0]
  ⋮               => ⋮

In [ ]:
for mj in 1:2
    nodes = Array{String}(undef, 50)
    for i in 1:length(nodes)
        node = node0
        for j in 1:mj
            if haskey(tree, node)
                node = sample_next_node2(payoffs, tree, node, 10.0)
            end
        end
        nodes[i] = node
    end
    nodes = sort(unique(nodes))
    for node in nodes
        display_node(payoffs, tree, node)
        println()
    end
end

## Play vs MCTS
### CPU first

In [370]:
n_mc = 100000
beta = 20.0
game = ""
tree = create_tree(game)
display_node(payoffs, create_tree(game), game)

Node: 
Stats: 0/0=NaN


In [371]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

 60.483277 seconds (559.44 M allocations: 42.634 GiB, 37.23% gc time)
Node: V
132112 {0}. 
Stats: 0/0=NaN


In [372]:
# player turn
pmove = int2ind[313111]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: Vi
132112 {0}. 313111 {0}. 
Stats: 0/0=NaN


In [373]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

 59.409009 seconds (505.02 M allocations: 40.165 GiB, 36.85% gc time)
Node: Vif
132112 {0}. 313111 {0}. 
312112 {0}. 
Stats: 0/0=NaN


In [374]:
# player turn
pmove = int2ind[131212]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: VifS
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
Stats: 0/0=NaN


In [375]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

 23.942032 seconds (174.92 M allocations: 14.672 GiB, 40.37% gc time)
Node: VifSU
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 
Stats: 0/0=NaN


In [376]:
# player turn
pmove = int2ind[112321]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: VifSU6
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
Stats: 0/0=NaN


In [377]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

  8.113058 seconds (72.07 M allocations: 5.766 GiB, 39.43% gc time)
Node: VifSU6;
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
113212 {3}. 
Stats: 0/0=NaN


In [378]:
# player turn
pmove = int2ind[113311]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: VifSU6;=
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
113212 {3}. 113311 {3}. 
Stats: 0/0=NaN


In [379]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

  8.334637 seconds (75.95 M allocations: 6.597 GiB, 32.82% gc time)
Node: VifSU6;=E
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
113212 {3}. 113311 {3}. 
121222 {3}. 
Stats: 0/0=NaN


In [380]:
# player turn
pmove = int2ind[111412]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: VifSU6;=E+
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
113212 {3}. 113311 {3}. 
121222 {3}. 111412 {3}. 
Stats: 0/0=NaN


In [381]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

 12.023327 seconds (100.81 M allocations: 9.897 GiB, 33.77% gc time)
Node: VifSU6;=E+"
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
113212 {3}. 113311 {3}. 
121222 {3}. 111412 {3}. 
111133 {4}. 
Stats: 0/0=NaN


In [382]:
display_node(payoffs, tree, game)

Node: VifSU6;=E+"
132112 {0}. 313111 {0}. 
312112 {0}. 131212 {2}. 
131311 {2}. 112321 {3}. 
113212 {3}. 113311 {3}. 
121222 {3}. 111412 {3}. 
111133 {4}. 
Stats: 58206/58206=1.0
[terminal] 


In [317]:
node = game[1:7]
display_node(payoffs, create_tree(node), node)

Node: icHRG"&
313111 {0}. 214111 {1}. 
121321 {1}. 131122 {2}. 
121312 {2}. 111133 {2}. 
111232 {2}. 
Stats: 0/0=NaN


In [318]:
tree = create_tree(node)
@time tree = grow_tree(payoffs, tree, node, n_mc)

  8.971087 seconds (75.64 M allocations: 6.939 GiB, 35.43% gc time)


Dict{String, Vector{Int64}} with 18936 entries:
  "icHRG\"&/bJa"    => [1, 1, 0]
  "icHRG\"&[\\h0"   => [27, 28, 0]
  "icHRG\"&/%Z:)"   => [2, 3, 0]
  "icHRG\"&f1)]"    => [3, 4, 0]
  "icHRG\"&+.h["    => [5, 5, 0]
  "icHRG\"&j25/i`"  => [26, 26, 1]
  "icHRG\"&+%ZJi"   => [1, 1, 0]
  "icHRG\"&g]/bJ"   => [0, 1, 0]
  "icHRG\"&fZh]i8"  => [2, 2, 1]
  "icHRG\"&fZg"     => [42, 47, 0]
  "icHRG\"&+[/acK"  => [3, 4, 0]
  "icHRG\"&g4+/"    => [8, 8, 0]
  "icHRG\"&f45Ji`"  => [4, 4, 1]
  "icHRG\"&fd[]i"   => [10, 10, 0]
  "icHRG\"&g:*1+]"  => [1, 1, 0]
  "icHRG\"&)cdaF"   => [1, 1, 0]
  "icHRG\"&+%j\\hJ" => [1, 1, 0]
  "icHRG\"&fJ[0i8"  => [6, 6, 1]
  "icHRG\"&g[/bcKE" => [0, 1, 0]
  "icHRG\"&*#ZJi"   => [1, 1, 0]
  "icHRG\"&+\\h05`" => [2, 2, 1]
  "icHRG\"&+%)8"    => [2, 2, 1]
  "icHRG\"&*J[]/1"  => [1, 1, 0]
  "icHRG\"&5%i`"    => [26, 26, 1]
  "icHRG\"&)[/^b"   => [3, 3, 0]
  ⋮                 => ⋮

### MCTS vs self

In [356]:
n_mc = 100000
beta = 20.0
game = ""
tree = create_tree(game)
display_node(payoffs, tree, game)

Node: 
Stats: 0/0=NaN


In [357]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 56.583590 seconds (558.71 M allocations: 42.540 GiB, 34.94% gc time)
Node: `
213112 {0}. 
Stats: 813/1705=0.476


In [358]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 54.774908 seconds (495.92 M allocations: 38.330 GiB, 37.49% gc time)
Node: `i
213112 {0}. 313111 {0}. 
Stats: 4529/11739=0.385


In [359]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 58.743900 seconds (510.55 M allocations: 40.618 GiB, 36.95% gc time)
Node: `ih
213112 {0}. 313111 {0}. 
312211 {0}. 
Stats: 241/677=0.355


In [360]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 26.903677 seconds (234.21 M allocations: 18.199 GiB, 36.70% gc time)
Node: `ihU
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
Stats: 3833/142574=0.026


In [361]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 14.737715 seconds (129.20 M allocations: 10.370 GiB, 34.92% gc time)
Node: `ihUX
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 
Stats: 1800/63040=0.028


In [362]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 13.172859 seconds (111.80 M allocations: 8.985 GiB, 35.84% gc time)
Node: `ihUX<
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
Stats: 188/4048=0.046


In [363]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 14.017991 seconds (121.78 M allocations: 10.367 GiB, 32.63% gc time)
Node: `ihUX<R
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 
Stats: 1349/103227=0.013


In [364]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 13.121446 seconds (111.25 M allocations: 9.680 GiB, 33.92% gc time)
Node: `ihUX<Rd
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
Stats: 150/2233=0.067


In [365]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 16.123942 seconds (132.70 M allocations: 12.364 GiB, 32.35% gc time)
Node: `ihUX<Rd\
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
212131 {3}. 
Stats: 294/27586=0.010


In [367]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 10.299981 seconds (86.84 M allocations: 7.930 GiB, 34.29% gc time)
Node: `ihUX<Rd\>
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
212131 {3}. 114112 {5}. 
Stats: 0/12978=0.0
[terminal] 


In [366]:
node = "<Bi;HR\"`"
display_node(payoffs, create_tree(node), node)

Node: <Bi;HR"`
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
Stats: 0/0=NaN


In [355]:
tree = create_tree(node)
@time tree = grow_tree(payoffs, tree, node, n_mc)
display_choices(payoffs, tree, node)

  5.259248 seconds (51.59 M allocations: 4.324 GiB, 32.27% gc time)
Node: <Bi;HR"`
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
Stats: 199854/199905=0.999

Node: <Bi;HR"`D
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
121213 {4}. 
Stats: 99894/99894=1.0
[terminal] 

Node: <Bi;HR"`G
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
121312 {3}. 
Stats: 64/115=0.556

Node: <Bi;HR"`J
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
122113 {4}. 
Stats: 99896/99896=1.0
[terminal] 


In [341]:
display_choices(payoffs, tree, "b`HG>ii")

Node: b`HG>ii
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 
Stats: 5294/13927=0.380

Node: b`HG>ii!
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111124 {0}. 
Stats: 54/93=0.580

Node: b`HG>ii"
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111133 {0}. 
Stats: 116/308=0.376

Node: b`HG>ii#
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111142 {0}. 
Stats: 71/145=0.489

Node: b`HG>ii$
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111214 {0}. 
Stats: 46/72=0.638

Node: b`HG>ii%
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111223 {0}. 
Stats: 58/104=0.557

Node: b`HG>ii&
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111232 {0}. 
Stats: 90/209=0.430

Node: b`HG>ii'
213211 {0}. 213112 {0}. 
121321 {0}. 121312

## Mate in X analysis

In [623]:
(2,4)

(2, 4)

In [695]:
struct Gamestate
    node::String
    children::Array{String}
    value::Int64
    terminal::Bool
end

function mate_checker(
        payoffs::Array{Int64},  
        node00::String;
        nits_tree::Int64 = 1000,
        nits_cands::Int64 = 1000,
        verbose::Bool = true
        )::Tuple{Float64, Int64}
    tree = create_tree(node00)
    tree = grow_tree(payoffs, tree, node00, nits_tree)
    # Range of turn nos.
    lens = [length(k) for k in keys(tree)]
    minlen = minimum(lens)
    maxlen = maximum(lens)
    nodes_by_length = Array{Array{String}}(undef, maxlen-minlen+1)
    for l in minlen:maxlen
        nodes_by_length[l-minlen+1] = [k for k in keys(tree) if length(k)==l]
    end
    # compute game values
    game_values = Dict(node00 => 0.0)
    for l in 1:length(nodes_by_length)
        ind = length(nodes_by_length)-l+1 # index of target length
        len = maxlen - l + 1
        wvalue = -1.0 # if len is even and node is terminal, then pl 1 loses
        if mod(len, 2)==1
            wvalue = 1.0
        end
        for node in nodes_by_length[ind]
            if tree[node][3]==1
                game_values[node]=wvalue
            else
                children=get_children(payoffs, node)
                val = 2 * ((tree[node][1]+1)/(tree[node][2]+2) - 0.5)
                for c in children
                    if haskey(tree, c)
                        v = game_values[c]
                        if mod(len, 2)==1
                            # player 2 to move: get minimum of children values
                            if v <= val
                                val = v
                            end
                        else
                            # player 1 to move: get maximum of children values
                            if v >= val
                                val = v
                            end
                        end
                    end
                end
                game_values[node] = val
            end
        end
    end
    # check length of best resistance
    best_res_length = Dict{String,Int64}()
    val = game_values[node00]
    if val > 0
        resister = 1
    else
        resister = 0 # player 2
    end
    for iter in 1:nits_cands
        node = node00
        val = game_values[node00]
        flag = true
        while flag
            cands = [c for c in keys(game_values) if length(c)==(length(node)+1) && c[1:length(node)]==node && game_values[c]==val]
            if length(cands) > 0
                node = rand(cands)
            else
                flag = false
            end
        end
        best_res_length[node] = length(node)-minlen
    end
    # Range of BRs
    lens_br = [length(k) for k in keys(best_res_length)]
    minlen_br = minlen
    maxlen_br = maximum(lens_br)
    for len in maxlen_br:-1:minlen_br
        nodes = unique([node[1:len] for node in keys(best_res_length) if length(node) >= len])
        for node in nodes
            if haskey(best_res_length, node)
                # do nothing
            else
                children = [c for c in keys(best_res_length) if length(c) == (length(node)+1) && c[1:length(node)]==node]
                brs = [best_res_length[c] for c in children]
                if mod(length(node), 2)==resister
                    # current node is resister to play
                    best_res_length[node] = maximum(brs)
                else
                    best_res_length[node] = minimum(brs)                
                end
            end
        end
    end
    if verbose
        node = node00
        br = best_res_length[node00]
        flag = true
        while flag
            cands = [nd for nd in keys(best_res_length) if length(nd)==(length(node)+1) && best_res_length[nd]==br && nd[1:length(node)]==node]
            if length(cands)==0
                flag = false
            else
                node = rand(cands)
            end
        end
        display_path(payoffs, tree, node)
    end
    return (game_values[node00], best_res_length[node00])
end

mate_checker (generic function with 4 methods)

In [696]:
# a game where P1 lost
node0 = "`ihUX<Rd\\>"
display_node(payoffs, create_tree(node0), node0)

Node: `ihUX<Rd\>
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
212131 {3}. 114112 {5}. 
Stats: 0/0=NaN


In [697]:
# find where a mate in X occurs
node00 = node0[1:4]
display_node(payoffs, create_tree(node00), node00)

Node: `ihU
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
Stats: 0/0=NaN


In [698]:
@time mate_checker(payoffs, node00, verbose=true)

Node: `ihU
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
Stats: 134/1425=0.094

  0.634322 seconds (3.84 M allocations: 249.702 MiB, 52.26% compilation time)


(-0.810791871058164, 0)

#### debugging code

In [699]:
node00 = "("
tree = create_tree(node00)
@time tree = grow_tree(payoffs, tree, node00, 10000)
display_node(payoffs, tree, node00)

  5.533680 seconds (47.59 M allocations: 3.683 GiB, 39.87% gc time)
Node: (
111313 {0}. 
Stats: 4352/10000=0.435


In [700]:
leaf = random_leaf(payoffs, tree, node00)
display_path(payoffs, tree, leaf)

Node: (
111313 {0}. 
Stats: 4352/10000=0.435

Node: (h
111313 {0}. 312211 {1}. 
Stats: 55/121=0.454

Node: (h@
111313 {0}. 312211 {1}. 
114211 {1}. 
Stats: 2/3=0.666



In [701]:
# Check mate-in-X property
# Range of turn nos.
lens = [length(k) for k in keys(tree)]
minlen = minimum(lens)
maxlen = maximum(lens)
nodes_by_length = Array{Array{String}}(undef, maxlen-minlen+1)
for l in minlen:maxlen
    nodes_by_length[l-minlen+1] = [k for k in keys(tree) if length(k)==l]
end

In [702]:
game_values = Dict(node00 => 0.0)
for l in 1:length(nodes_by_length)
    ind = length(nodes_by_length)-l+1 # index of target length
    len = maxlen - l + 1
    wvalue = -1.0 # if len is even and node is terminal, then pl 1 loses
    if mod(len, 2)==1
        wvalue = 1.0
    end
    for node in nodes_by_length[ind]
        if tree[node][3]==1
            game_values[node]=wvalue
        else
            children=get_children(payoffs, node)
            val = 2 * ((tree[node][1]+1)/(tree[node][2]+2) - 0.5)
            for c in children
                if haskey(tree, c)
                    v = game_values[c]
                    if mod(len, 2)==1
                        # player 2 to move: get minimum of children values
                        if v <= val
                            val = v
                        end
                    else
                        # player 1 to move: get maximum of children values
                        if v >= val
                            val = v
                        end
                    end
                end
            end
            game_values[node] = val
        end
    end
end

In [703]:
game_values

Dict{String, Float64} with 17590 entries:
  "(8'E"   => -0.333333
  "(NR\$"  => 0.333333
  "(#'L"   => 0.333333
  "(360"   => 0.333333
  "(`PaS"  => -0.333333
  "(:("    => -0.333333
  "(*,"    => -0.333333
  "(1G"    => 0.333333
  "(*>"    => -0.333333
  "(icS"   => -0.333333
  "(PIi"   => 0.333333
  "(cBHC"  => -0.333333
  "(U&G"   => -0.333333
  "(;Hh"   => -0.333333
  "(H*I"   => -0.333333
  "(\"]"   => -0.333333
  "(2bL"   => -0.333333
  "(i?@N"  => 0.333333
  "()*>"   => -0.333333
  "(W'"    => -0.333333
  "(fOL"   => 0.333333
  "(ZU&"   => 0.333333
  "(J>)"   => 0.333333
  "(\"7\"" => -0.333333
  "(MX"    => 0.333333
  ⋮        => ⋮

In [704]:
l = 1
for node in nodes_by_length[l]
    display_node(payoffs, tree, node)
    println(game_values[node])
    println()
end

Node: (
111313 {0}. 
Stats: 4352/10000=0.435
-0.33333333333333337



In [705]:
# check length of best resistance
best_res_length = Dict{String,Int64}()
val = game_values[node00]
if val > 0
    resister = 1
else
    resister = 0 # player 2
end

0

In [706]:
for iter in 1:100
    node = node00
    val = game_values[node00]
    flag = true
    while flag
        cands = [c for c in keys(game_values) if length(c)==(length(node)+1) && c[1:length(node)]==node && game_values[c]==val]
        if length(cands) > 0
            node = rand(cands)
        else
            flag = false
        end
    end
    best_res_length[node] = length(node)-minlen
end
best_res_length

Dict{String, Int64} with 89 entries:
  "(cT-S"  => 4
  "(`b=E"  => 4
  "(H`GZ"  => 4
  "(`BR\"" => 4
  "(H=\\_" => 4
  "(ceS^"  => 4
  "(ibLU"  => 4
  "(iTb>"  => 4
  "(H=Q?"  => 4
  "(`>YU"  => 4
  "(H`@\\" => 4
  "(H*=["  => 4
  "(HC)i"  => 4
  "(`;J8"  => 4
  "(`>2P"  => 4
  "(`BC9"  => 4
  "(H=^0"  => 4
  "(ibHR"  => 4
  "(`SLW"  => 4
  "(`>E2"  => 4
  "(c@-R"  => 4
  "(cI%'"  => 4
  "(cQ\"#" => 4
  "(cR>F"  => 4
  "(iR&4"  => 4
  ⋮        => ⋮

In [707]:
# Range of BRs
lens_br = [length(k) for k in keys(best_res_length)]
minlen_br = minlen
maxlen_br = maximum(lens_br)

5

In [708]:
for len in maxlen_br:-1:minlen_br
    nodes = unique([node[1:len] for node in keys(best_res_length) if length(node) >= len])
    for node in nodes
        if haskey(best_res_length, node)
            # do nothing
        else
            children = [c for c in keys(best_res_length) if length(c) == (length(node)+1) && c[1:length(node)]==node]
            brs = [best_res_length[c] for c in children]
            if mod(length(node), 2)==resister
                # current node is resister to play
                best_res_length[node] = maximum(brs)
            else
                best_res_length[node] = minimum(brs)                
            end
        end
    end
end

In [709]:
# node = node00
# val = game_values[node00]
# flag = true
# while flag
#     cands = [c for c in nodes_by_length[length(node)-minlen+2] if c[1:length(node)]==node && game_values[c]==val]
#     if length(cands) > 0
#         node = rand(cands)
#     else
#         flag = false
#     end
# end
# display_path(payoffs, tree, node)

In [710]:
[nd => best_res_length[nd] for nd in keys(best_res_length) if length(nd)==1]

1-element Vector{Pair{String, Int64}}:
 "(" => 4

In [711]:
game_values[node00]

-0.33333333333333337

### automate finding mate-in-2 problems

In [ ]:
n_mc = 10000
beta = 20.0
game = ""
tree = create_tree(game)
flag = true
while flag
    # MCTS turn
    tree = create_tree(game)
    tree = grow_tree(payoffs, tree, game, n_mc)
    game = sample_next_node2(payoffs, tree, game, beta)
    #display_node(payoffs, tree, game)
    value, br = mate_checker(payoffs, game, nits_tree = n_mc, verbose=false)
    if abs(value)==1.0
        flag = false
        println("----MATE FOUND---")
        value, br = mate_checker(payoffs, game, nits_tree = n_mc, verbose=true)
        print("value:")
        print(value)
        print(" best resist:")
        print(br)
    end
    if tree[game][3]==1
        flag = false
    end
end